#### Loading notebook
###### Code Ocean use only

In [5]:

%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import os

import glob
import re
from datetime import datetime
import sys
from pathlib import Path
from typing import Union, Dict, List, Optional, Tuple
import tqdm

from aind_dynamic_foraging_data_utils import nwb_utils as nu
import aind_dynamic_foraging_basic_analysis.licks.annotation as a
# Add this at the top of your notebook
import sys
sys.path.insert(0, '/root/capsule/src/utils')
import behavior_utils as bu
import array_utils as au
import load_utils as lu

ModuleNotFoundError: No module named 'numpy'

In [58]:

import sys
sys.path.insert(0, '/Users/nickkeesey/Desktop/learning-dynamics-behavior')

# Then your original import should work
import test_load as tl
import process_nwbs as pn

##### Full Processing Notebook: 
###### 1. Use nwb_utils to load nwbs from base directory and create df_trials 
###### 2. Use session_metrics to compute session metadata and performance metrics from df_trials
###### 3. Use download_utils to download the processed DataFrame to a specified filepath


____
##### Full Session Processing:

##### Single Session Testing: 

In [27]:
# Set CO atttached data asset as base directory of nwbs to be processed
base_dir='/root/capsule/data/foraging_nwb_bonsai'

# Set download location for combined DataFrame
destination_path = '/root/capsule/data/file_process_output'

In [83]:
nwb = nu.load_nwb_from_filename('/Users/nickkeesey/Desktop/learning-dynamics-behavior/data/769887_2024-11-06_09-39-22.nwb')

df_trials = pn.compute_df_trial(nwb)

meta_df = pn.compute_df_session_meta(nwb, df_trials)
meta_df.columns = meta_df.columns.droplevel(0) # Remove 'metadata' level

performance_df = pn.compute_df_session_performance(nwb, df_trials)
performance_df.columns = performance_df.columns.droplevel(0) # Remove 'session_stats' level

# Convert MultiIndex to flat index before concatenation
if isinstance(meta_df.index, pd.MultiIndex):
    meta_df = meta_df.reset_index()
if isinstance(performance_df.index, pd.MultiIndex):
    performance_df = performance_df.reset_index()

# Now concatenate the dataframes
session_df = pd.concat([meta_df, performance_df], axis=1)


In [84]:
# Check with real data
df_compare = pd.read_csv('/Users/nickkeesey/Desktop/learning-dynamics-behavior/data/filtered_data_20241107.csv')

subject = 769887
session_date = '2024-11-06'

# Flatten the MultiIndex columns in session_df
if isinstance(session_df.columns, pd.MultiIndex):
    session_df.columns = session_df.columns.get_level_values(-1)

df_compare = df_compare[df_compare['subject_id'] == subject]
df_compare = df_compare[df_compare['session_date'] == session_date]

# Get columns unique to each dataframe
df_cols = set(session_df.columns)
df_compare_cols = set(df_compare.columns)

print("Columns in df but not in df_compare:")
print(df_cols - df_compare_cols)
print("\nColumns in df_compare but not in df:")
print(df_compare_cols - df_cols)

def is_number(value):
    try:
        float(value)
        return True
    except (TypeError, ValueError):
        return False

def is_close_enough(val1, val2, sig_figs=4):
    if not (is_number(val1) and is_number(val2)):
        return val1 == val2
    
    # Convert to floats
    val1, val2 = float(val1), float(val2)
    
    # Handle zero values
    if val1 == 0 or val2 == 0:
        return abs(val1 - val2) < 1e-10
    
    # Calculate relative difference
    rel_diff = abs((val1 - val2) / max(abs(val1), abs(val2)))
    return rel_diff < 10**(-sig_figs)

# Compare values for common columns
common_cols = df_cols.intersection(df_compare_cols)
print("\nComparing values for common columns:")
for col in common_cols:
    session_val = session_df[col].iloc[0]
    df_compare_val = df_compare[col].iloc[0]
    
    if pd.isna(session_val) and pd.isna(df_compare_val):
        print(f"{col}: Both NaN")
    elif is_close_enough(session_val, df_compare_val):
        print(f"{col}: Same value (within 4 sig figs)")
        print(f"  session_df: {session_val}")
        print(f"  df_compare: {df_compare_val}")
    else:
        print(f"{col}: Different values")
        print(f"  session_df: {session_val}")
        print(f"  df_compare: {df_compare_val}")

Columns in df but not in df_compare:
set()

Columns in df_compare but not in df:
{'logistic_Hattori2019_score_std', 'logistic_Hattori2019_bias', 'logistic_Bari2019_score_std', 'session_time', 'logistic_Miller2021_Choice_amp', 'logistic_Bari2019_bias', 'virus', 'logistic_Su2022_RewC_tau', 'logistic_Miller2021_Choice_x_Reward_amp', 'logistic_Miller2021_score_std', 'subject_genotype', 'logistic_Miller2021_Reward_amp', 'results', 'logistic_Bari2019_RewC_amp', 'logistic_Miller2021_Choice_tau', 'abs(logistic_Miller2021_bias)', 'data_source', 'has_ephys', 'logistic_Hattori2019_score_mean', 'logistic_Hattori2019_UnrC_tau', 'hardware', 'abs(logistic_Hattori2019_bias)', 'logistic_Miller2021_Choice_x_Reward_tau', 'avg_trial_length_in_seconds', 'weekday', 'logistic_Miller2021_bias', 'session_name', 'task_type', 'logistic_Hattori2019_RewC_amp', 'logistic_Hattori2019_RewC_tau', 'logistic_Su2022_score_std', 'injections', 'logistic_Bari2019_Choice_amp', 'water_after_session_last_session', 'old_bpod_se

/var/folders/mr/k7gx9xps333170k7zsggl4tw0000gn/T/ipykernel_61423/489206852.py:2: DtypeWarning: Columns (6,7,8,11,26,27,28,29,30,133,134,149,150,151,152,153,154,155,156,157,158,159,160,161) have mixed types. Specify dtype option on import or set low_memory=False.
  df_compare = pd.read_csv('/Users/nickkeesey/Desktop/learning-dynamics-behavior/data/filtered_data_20241107.csv')


##### Full Bonsai Processing:

In [ ]:
def process_nwb_to_df(nwb_path, nwb_utils, process_nwbs):
    """Process a single NWB file into a dataframe containing both meta and performance data
    
    Args:
        nwb_path (str): Path to NWB file
        nwb_utils: Module containing load_nwb_from_filename function
        process_nwbs: Module containing compute functions
        
    Returns:
        pd.DataFrame: Combined dataframe with meta and performance data
    """
    try:
        # Load NWB file using provided utility
        nwb = nwb_utils.load_nwb_from_filename(nwb_path)
        
        # Compute trial, meta and performance dataframes
        df_trials = process_nwbs.compute_df_trial(nwb)
        
        meta_df = process_nwbs.compute_df_session_meta(nwb, df_trials)
        meta_df.columns = meta_df.columns.droplevel(0)  # Remove 'metadata' level
        
        performance_df = process_nwbs.compute_df_session_performance(nwb, df_trials) 
        performance_df.columns = performance_df.columns.droplevel(0)  # Remove 'session_stats' level
        
        # Convert MultiIndex to flat index before concatenation
        if isinstance(meta_df.index, pd.MultiIndex):
            meta_df = meta_df.reset_index()
        if isinstance(performance_df.index, pd.MultiIndex):
            performance_df = performance_df.reset_index()
            
        # Concatenate the dataframes
        session_df = pd.concat([meta_df, performance_df], axis=1)
        
        return session_df
    
    except Exception as e:
        print(f"Error processing {nwb_path}: {str(e)}")
        return None

def process_multiple_nwbs(nwb_dir, nwb_utils, process_nwbs, pattern="**/*.nwb"):
    """Process multiple NWB files from a directory and combine into one dataframe
    
    Args:
        nwb_dir (str): Directory containing NWB files
        nwb_utils: Module containing load_nwb_from_filename function
        process_nwbs: Module containing compute functions
        pattern (str): Glob pattern to match NWB files. Default "**/*.nwb"
        
    Returns:
        pd.DataFrame: Combined dataframe with data from all NWB files
    """
    # Get list of all NWB files
    nwb_files = glob.glob(os.path.join(nwb_dir, pattern), recursive=True)
    
    if not nwb_files:
        raise ValueError(f"No NWB files found in {nwb_dir} matching pattern {pattern}")
        
    # Process each NWB file and collect results
    all_dfs = []
    for nwb_path in tqdm(nwb_files, desc="Processing NWB files"):
        df = process_nwb_to_df(nwb_path, nwb_utils, process_nwbs)
        if df is not None:
            all_dfs.append(df)
            
    # Combine all dataframes
    if not all_dfs:
        raise ValueError("No valid dataframes were generated from the NWB files")
        
    combined_df = pd.concat(all_dfs, axis=0, ignore_index=True)
    
    return combined_df

# Example usage:
if __name__ == "__main__":
    import nwb_utils as nu  # Module containing load_nwb_from_filename
    import process_nwbs as pn  # Module containing compute functions
    
    # Directory containing NWB files
    nwb_directory = "/root/capsule/data/foraging_nwb_bonsai"
    
    try:
        # Process all NWB files and get combined dataframe
        combined_df = process_multiple_nwbs(nwb_directory, nu, pn)
        
        # Save to CSV or pickle
        combined_df.to_csv("combined_session_data.csv", index=False)
        
        print(f"Successfully processed {len(combined_df)} sessions")
        
    except Exception as e:
        print(f"Error in pipeline: {str(e)}")

____
##### 1. Trial Only Processing with subject_id and session_date columns

In [30]:
# Load nwbs from base directory with filenames matching valid regex patterns 
# Returns csv files into scratch folder (default)
lu.load_nwb_files(base_dir = base_dir)

OSError: [Errno 30] Read-only file system: '/root'

In [23]:
# Set source path for combined Pandas DataFrame creation
source_path = '/root/capsule/scratch'

# Create concatenated DataFrame using CSV files in source path
# Add subject_id and session_date categorical columns based on filename
processed_nwb_df = lu.process_csv_files(source_path)
print(processed_nwb_df.shape) 

Could not extract metadata from 0_2024-11-21.csv
Could not extract metadata from 1_2024-09-30.csv
Could not extract metadata from 45678_2024-09-11.csv
Could not extract metadata from 1_2024-04-13.csv
Could not extract metadata from 1_2024-09-16.csv
Could not extract metadata from 3_2024-09-23.csv
Could not extract metadata from 23456789_2024-10-31.csv
Could not extract metadata from 324567890_2024-10-31.csv
Could not extract metadata from 3_2024-10-21.csv
Could not extract metadata from 5_2024-09-09.csv
Could not extract metadata from 0_2024-12-04.csv
Could not extract metadata from 2_2024-08-14.csv
Could not extract metadata from 2_2024-08-05.csv
Could not extract metadata from 324567890_2024-11-01.csv
Could not extract metadata from 1_2024-04-19.csv
Could not extract metadata from 3_2024-10-07.csv
Could not extract metadata from 1_2024-04-06.csv
Could not extract metadata from 1_2024-08-19.csv
Could not extract metadata from 2345678_2024-11-01.csv
Could not extract metadata from 0_20

,trial,animal_response,rewarded_historyL,rewarded_historyR,bait_left,bait_right,base_reward_probability_sum,reward_probabilityL,reward_probabilityR,reward_random_number_left,...,reward_time_in_session,reward_time_in_trial,choice_time_in_session,choice_time_in_trial,earned_reward,extra_reward,subject_id,session_date,reward_delay,minimum_opto_interval
0,0,2.0,False,False,False,False,0.45,0.4,0.05,0.789076,...,NaN,NaN,NaN,NaN,False,False,687553,2023-12-01,NaN,NaN
1,1,2.0,False,False,False,False,0.45,0.4,0.05,0.518192,...,NaN,NaN,NaN,NaN,False,False,687553,2023-12-01,NaN,NaN
2,2,0.0,True,False,True,False,0.45,0.4,0.05,0.367511,...,16.160096,0.487072,16.153088,0.480064,True,False,687553,2023-12-01,NaN,NaN
3,3,0.0,True,False,True,False,0.45,0.4,0.05,0.053372,...,25.333088,0.376064,25.331872,0.374848,True,False,687553,2023-12-01,NaN,NaN
4,4,0.0,False,False,False,False,0.45,0.4,0.05,0.955191,...,NaN,NaN,34.364160,0.157440,False,False,687553,2023-12-01,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2577768,582,1.0,False,False,False,False,0.45,0.4,0.05,0.700221,...,NaN,NaN,5378.042624,0.199680,False,False,724555,2024-07-10,0.0,0.0
2577769,583,2.0,False,False,False,False,0.45,0.4,0.05,0.678467,...,NaN,NaN,NaN,NaN,False,False,724555,2024-07-10,0.0,0.0
2577770,584,1.0,False,False,True,False,0.45,0.4,0.05,0.050451,...,NaN,NaN,5392.707232,0.347424,False,False,724555,2024-07-10,0.0,0.0
2577771,585,1.0,False,False,True,False,0.45,0.4,0.05,0.118763,...,NaN,NaN,5404.919264,0.359232,False,False,724555,2024-07-10,0.0,0.0


In [24]:
# Set download location for combined DataFrame
filepath = '/root/capsule/data/file_process_output'

# Download DataFrame to selected filepath 
downloaded_path = lu.download_dataframe_to_csv(
    processed_nwb_df, 
    filepath=filepath, 
    filename='foraging_trials_bonsai.csv', 
    overwrite=True
)

In [2]:
# Check downloaded size 

file_size = os.path.getsize(filepath)
print("File size:", file_size, "bytes")

File size: 40 bytes
